This notebook can be used to run the simulation instead of having to type the commands into the command line interface

In [1]:
# Copyright (c) 2021 The MATCH Authors. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

import multiprocessing as mp
import os
from pathlib import Path
import match_model.generate_input_files as generate_input_files
import shutil
import time

# Enter the path to the folder where your model inputs are located

In [2]:
# specify the location of the model runs folder you will be using, either as a relative filepath or an absolute filepath
# the default location is the MODEL_RUNS directory within the Git repo
# however, if you want to save model runs in a alternate location, such as a Box folder, you could use something like:

model_run_location = '../../MODEL_RUNS/test'
#model_run_location = 'C:/Users/gmiller/GitHub/SWITCH247/MODEL_RUNS/test'
#model_run_location = '../../../../../Public/MODEL_RUNS/run_19_22_test'

In [3]:
#ensure that the filepath string uses forward slashes rather than backslashes
model_run_location = model_run_location.replace('\\','/')

# if the location is a relative filepath
if '../' in model_run_location:
    model_workspace = Path.cwd() / model_run_location
# otherwise if it is an absoluate filepaht
elif ':/' in model_run_location:
    model_workspace = Path(model_run_location)

# Generate and Validate model inputs

The following cell will load all of the input data from the `model_inputs.xlsx` file in your run folder, validate the inputs, and create all of the required input files in the format required by the solver. 

This cell will perform the following steps:  
1. If input files do not yet exist, create them
2. If input files exist, check that the current software version matches the version that was used to generate the inputs
3. If the versions do not match, re-generate the input files to ensure consistency

In [4]:
# check if the directory exists
if os.path.exists(model_workspace / 'inputs'):
    # check that an inputs version file exists
    if os.path.exists(model_workspace / 'inputs_version.txt'):
        # get the version number
        with open(model_workspace / 'inputs_version.txt', 'r') as i:
            inputs_version = i.read()
        # get the current version number
        version_path = Path.cwd() / 'version.py'
        version = {}
        with open(version_path) as f:
            exec(f.read(), version)
        version = version['__version__']
        # check if the versions match
        if version == inputs_version:
            print(f'Input files already generated with current software version ({version})')
        else:
            print(f'Inputs were generated using version {inputs_version}. Current version is {version}')
            print('Removing old files and re-generating inputs with current version. Please wait...')
            old_files = os.listdir(model_workspace)
            old_files.remove('model_inputs.xlsx')
            for f in old_files:
                try:
                    os.remove(model_workspace / f)
                except PermissionError:
                    shutil.rmtree(model_workspace / f)
            generate_input_files.generate_inputs(model_workspace)
    else: 
        generate_input_files.generate_inputs(model_workspace)
# if the directory does not exist, generate the input files
else:
    print('Generating inputs now...')
    generate_input_files.generate_inputs(model_workspace)

Generating inputs now...
Loading data from excel spreadsheet...


C:\Users\gmiller\.conda\envs\match_model\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Writing options.txt...
Creating input and output folders for each scenario...


C:\Users\gmiller\.conda\envs\match_model\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Generating capacity factor timeseries for run_19

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\Users\gmiller\GitHub\SWITCH247\match_model\notebooks\..\..\MODEL_RUNS\test\run_19\PySAM Downloaded Weather Files\Wind\windtoolkit_38.1475_-121.8519_60min_80m_2011.srw

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\Users\gmiller\GitHub\SWITCH247\match_model\notebooks\..\..\MODEL_RUNS\test\run_19\PySAM Downloaded Weather Files\Wind\windtoolkit_38.1475_-121.8519_60min_80m_2012.srw

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\Users\gmiller\GitHub\SWITCH247\match_model\notebooks\..\..\MODEL_RUNS\test\run_19\PySAM Downloaded Weather Files\Wind\windtoolkit_38.1475_-121.8519_60min_80m_2013.srw

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\Users\gmiller\GitHub\SWITCH247\match_model\noteboo

C:\Users\gmiller\.conda\envs\match_model\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


Writing inputs for tc_80 scenario...


C:\Users\gmiller\.conda\envs\match_model\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


# Solve Model Scenarios

### Option 1: Run as many parallel threads as possible

If you would like to run as many scenarios in parallel as your machine can support, run the following cell. 

This will identify how many processor cores (n) exist on your machine, and automatically run (n-1) scenarios (or the total number of scnearios, whichever is less). This should leave at least one core of your machine open for other tasks, but we still recommend that you close all other windows/processes before starting this option.

In [5]:
num_processors = mp.cpu_count()
print(f'This machine has {num_processors} CPU cores')
# get the number of scenarios to run
num_scenarios = len(os.listdir(model_workspace / 'inputs'))
print(f'There are {num_scenarios} scenarios to run')
# determine how many threads to open
parallel_threads = min(num_processors - 1, num_scenarios)
print(f'Running {parallel_threads} scenarios in parallel...')
i = 0
while i < parallel_threads:
    os.system(f'start cmd /k "cd {model_run_location} & activate match_model & match solve-scenarios"') 
    time.sleep(2)
    i += 1


This machine has 28 CPU cores
There are 2 scenarios to run
Running 2 scenarios in parallel...


## Option 2: Start scenarios one at a time

If you would like to only open a single solver window, run the following cell. Each time you run the following cell will open a new parallel solver.

In [ ]:
os.system(f'start cmd /k "cd {model_run_location} & activate match_model & match solve-scenarios"') 